## Accessing DM-EFD data


In this notebook we demonstrate how to extract data from the DM-EFD using [aioinflux](https://aioinflux.readthedocs.io/en/stable/index.html), a Python client for InfluxDB, and proceed with data analysis using Pandas dataframes. 

This is complementaty to the [Chronograf](https://test-chronograf-efd.lsst.codes) interface which we use for time-series visualization.

In addition to `aioinflux`, you'll need to install `pandas`, `numpy` and `matplotlib` to run this notebook.

In [1]:
import matplotlib
%matplotlib widget
from matplotlib import pylab as plt
import aioinflux
import getpass
import pandas as pd
import asyncio
import numpy as np

from bokeh.plotting import figure, output_notebook, show
from bokeh.models import LinearAxis, Range1d
output_notebook()

Loading BokehJS ...

We'll access the DM-EFD instance deployed at the AuxTel lab in Tucson. You need to be on site or connected to the NOAO VPN. 

If you are familiar with the AuxTel lab environment, you might be able to authenticate using the generic `saluser`. Ping me at Slack (`@afausti`) if you have any problem.

In [2]:
username = "saluser"
password = getpass.getpass(f"Password for {username}:")

Password for saluser: ········


The following configures the `aioinflux` Python client to connect to the DM-EFD InfluxDB instance. 

In [3]:
client = aioinflux.InfluxDBClient(host='summit-influxdb-efd.lsst.codes', 
                                  port='443', 
                                  ssl=True, 
                                  username=username, 
                                  password=password,
                                  db='efd')

We can configure the output to be a Pandas dataframe, which is very convenient for data analysis.  Specify a time range for data in `InfluxQL`.  The default is 20hrs ago, but this may need to be changed depending on how recently data was taken.

In [4]:
client.output = 'dataframe'

#time_span = "time >= '2019-12-11T04:20:00+00:00' AND time <= '2019-12-11T04:33:00+00:00'"  # UTC
time_span = "time >= '2019-12-12T05:21:00+00:00' AND time <= '2019-12-12T05:23:00+00:00'"  # 

time_span = "time >= '2019-12-14T02:17:20+00:00' AND time <= '2019-12-14T02:17:50+00:00'"  # Simon


# Had a stop tracking issue occur at 9:49pm

Query each of the measurements we may want to correlate later in the notebook.  Note that this could be done as a single query, but the result is a dictionary of `DataFrames` which I find less convenient to use than named variables corresponding to one `DataFrame` each.

In [5]:
async def get_data_frame(field_base, topic, els=100):
    fields = ", ".join([f'"{field_base}{i}"' for i in range(100)])
    df = await client.query(f'SELECT "cRIO_timestamp", "private_sndStamp", {fields} FROM "efd"."autogen"."{topic}" WHERE {time_span}')

    times = []
    timestamps = []
    vals = []
    step = 1./els
    for row in df.itertuples():
        for i in range(els):
            times.append(row.cRIO_timestamp + i*step)
            timestamps.append((pd.Timestamp(row.cRIO_timestamp, unit='s', tz='GMT') + pd.Timedelta(i*step, unit='s')))
            vals.append(getattr(row, f'{field_base}{i}'))

    return pd.DataFrame({'times':times, field_base:vals}, index=timestamps)

## Start with Azimuth

In [6]:
az = await get_data_frame('azimuthCalculatedAngle', 'lsst.sal.ATMCS.mount_AzEl_Encoders')
az_raw = await client.query(f'SELECT "azimuthCalculatedAngle99", "private_sndStamp", "private_rcvStamp" FROM "efd"."autogen"."lsst.sal.ATMCS.mount_AzEl_Encoders" WHERE {time_span}')

In [7]:
commanded_az = await get_data_frame('azimuth', 'lsst.sal.ATMCS.trajectory')
commanded_az_raw = await client.query(f'SELECT "azimuth99", "private_sndStamp", "private_rcvStamp" FROM "efd"."autogen"."lsst.sal.ATMCS.trajectory" WHERE {time_span}')

In [8]:
commanded_az_ATPng = await client.query(f'SELECT "azimuth", "private_sndStamp", "private_rcvStamp" FROM "efd"."autogen"."lsst.sal.ATMCS.command_trackTarget" WHERE {time_span}')

In [9]:
pd.to_datetime(commanded_az_ATPng['private_sndStamp'][0], unit='s')

Timestamp('2019-12-14 02:17:19.993353605')

### Plot commanded position (by pointing component), target position by ATMCS, Calculated position by ATMCS

In [10]:
yr_cen=np.median(commanded_az['azimuth'])
#yr_cen=0
dy=2.2
p = figure(x_axis_type='datetime', y_range=(yr_cen-dy, yr_cen+dy), plot_width=800, plot_height=400)
p.yaxis.axis_label = "Azimuth (degrees)"
p.xaxis.axis_label = "Time"
p.line(x=commanded_az.index.values, y=commanded_az['azimuth'], color='black', line_width=2, legend_label='ATMCS Commanded Az Trajectory')
p.cross(x=az.index.values, y=az['azimuthCalculatedAngle'], color='red', line_width=2, line_dash='dashed', legend_label='ATMCS Measured Az Position')
p.cross(x=pd.to_datetime(commanded_az_ATPng['private_sndStamp'], unit='s'), y=commanded_az_ATPng['azimuth'], color='green', line_width=2, legend_label='ATPng Target Az')

#p.cross(x=pd.to_datetime(commanded_az_raw['private_sndStamp'], unit='s'), y=commanded_az_raw['azimuth99'], color='black', line_width=2, legend_label='ATMCS Target Az')
#p.line(x=pd.to_datetime(az_raw['private_sndStamp'], unit='s'), y=az_raw['azimuthCalculatedAngle99'], color='lightblue', line_width=2, legend_label='Computed Az')


p.legend.location = 'bottom_left'
p.legend.click_policy = 'hide'
show(p)

## Look at motor velocities in azimuth

In [12]:
#Measured
measured_vel_az1 = await get_data_frame('azimuthMotor1Velocity', 'lsst.sal.ATMCS.measuredMotorVelocity')
measured_vel_az2 = await get_data_frame('azimuthMotor2Velocity', 'lsst.sal.ATMCS.measuredMotorVelocity')
# Commanded
commanded_vel_az = await get_data_frame('azimuthVelocity', 'lsst.sal.ATMCS.trajectory')
# From Pointing
commanded_vel_az_ATPng = await client.query(f'SELECT "azimuthVelocity", "private_sndStamp", "private_rcvStamp" FROM "efd"."autogen"."lsst.sal.ATMCS.command_trackTarget" WHERE {time_span}')

In [13]:
p = figure(x_axis_type='datetime', y_range=(yr_cen-dy, yr_cen+dy), plot_width=800, plot_height=400)
p.yaxis.axis_label = "Azimuth (degrees)"
p.xaxis.axis_label = "Time"
# Positions
p.line(x=commanded_az.index.values, y=commanded_az['azimuth'], color='black', line_width=2, legend_label='ATMCS Commanded Az Trajectory')
p.cross(x=az.index.values, y=az['azimuthCalculatedAngle'], color='red', line_width=2, line_dash='dashed', legend_label='ATMCS Measured Az Position')
p.cross(x=pd.to_datetime(commanded_az_ATPng['private_sndStamp'], unit='s'), y=commanded_az_ATPng['azimuth'], color='green', line_width=2, legend_label='ATPng Target Az')
# Velocities
p.extra_y_ranges = {'Velocity': Range1d(start=-0.1, end=0.1)}
p.add_layout(LinearAxis(y_range_name='Velocity', axis_label='Velocity'), 'right')
p.line(x=commanded_vel_az.index.values, y=commanded_vel_az['azimuthVelocity'], color='black', alpha=0.7, y_range_name='Velocity', legend_label='ATMCS Commanded Az Velocity Trajectory')
p.line(x=measured_vel_az1.index.values, y=measured_vel_az1['azimuthMotor1Velocity'], color='red', alpha=0.5, y_range_name='Velocity', legend_label='ATMCS Measured Az Velocity Motor 1')
p.line(x=measured_vel_az2.index.values, y=measured_vel_az2['azimuthMotor2Velocity'], color='blue', alpha=0.5, y_range_name='Velocity', legend_label='ATMCS Measured Az Velocity Motor 2')
p.x(x=pd.to_datetime(commanded_vel_az_ATPng['private_sndStamp'], unit='s'), y=commanded_vel_az_ATPng['azimuthVelocity'], y_range_name='Velocity', color='green', line_width=2, legend_label='ATPng Target Az Velocity')

p.legend.location = 'top_left'
p.legend.click_policy = 'hide'
show(p)

## Now look at the Elevation Axis

In [14]:
# Measured Position
el = await get_data_frame('elevationCalculatedAngle', 'lsst.sal.ATMCS.mount_AzEl_Encoders')

In [15]:
# Commanded Trajectory
commanded_el = await get_data_frame('elevation', 'lsst.sal.ATMCS.trajectory')

In [16]:
# Pointing Kernel Command
commanded_el_ATPng = await client.query(f'SELECT "elevation", "private_sndStamp", "private_rcvStamp" FROM "efd"."autogen"."lsst.sal.ATMCS.command_trackTarget" WHERE {time_span}')

In [17]:
# Looks like demanded Torques are too small to get measured...
#el_motor_torque_demand = await get_data_frame('elevationMotorTorque', 'lsst.sal.ATMCS.ATMCS_torqueDemand')
#el_motor_torque_measured = await get_data_frame('elevationMotorTorque', 'lsst.sal.ATMCS.ATMCS_measuredTorque')a

In [18]:
# Plot positions
yr_cen=np.median(commanded_el['elevation'])
#yr_cen=0
dy=1.1*(np.max(commanded_el['elevation'])-yr_cen)
p = figure(x_axis_type='datetime', y_range=(yr_cen-dy, yr_cen+dy), plot_width=800, plot_height=400)
p.yaxis.axis_label = "Nasmyth Position (degrees)"
p.xaxis.axis_label = "Time"
p.line(x=commanded_el.index.values, y=commanded_el['elevation'], color='black', line_width=2, legend_label='ATMCS Commanded Elevation Trajectory')
p.cross(x=el.index.values, y=el['elevationCalculatedAngle'], color='red', line_width=2, line_dash='dashed', legend_label='ATMCS Elevation Measured Nas1 Position')
p.cross(x=pd.to_datetime(commanded_el_ATPng['private_sndStamp'], unit='s'), y=commanded_el_ATPng['elevation'], color='green', line_width=2, legend_label='ATPng Target Elevation')

p.legend.location = 'bottom_left'
p.legend.click_policy = 'hide'
show(p)

## Look at Elevation motor velocities

In [19]:
#Measured
measured_vel_el = await get_data_frame('elevationMotorVelocity', 'lsst.sal.ATMCS.measuredMotorVelocity')

In [20]:
# Commanded
commanded_vel_el = await get_data_frame('elevationVelocity', 'lsst.sal.ATMCS.trajectory')

In [21]:
# From Pointing
commanded_vel_el_ATPng = await client.query(f'SELECT "elevationVelocity", "private_sndStamp", "private_rcvStamp" FROM "efd"."autogen"."lsst.sal.ATMCS.command_trackTarget" WHERE {time_span}')

In [22]:
# Plot Velocities and Positions
# Positions
p = figure(x_axis_type='datetime', y_range=(yr_cen-dy, yr_cen+dy), plot_width=800, plot_height=400)
p.yaxis.axis_label = "Elevation Position (degrees)"
p.xaxis.axis_label = "Time"
p.line(x=commanded_el.index.values, y=commanded_el['elevation'], color='black', line_width=2, legend_label='ATMCS Commanded Elevation Trajectory')
p.cross(x=el.index.values, y=el['elevationCalculatedAngle'], color='red', line_width=2, line_dash='dashed', legend_label='ATMCS Elevation Measured Elevation Position')
p.cross(x=pd.to_datetime(commanded_el_ATPng['private_sndStamp'], unit='s'), y=commanded_el_ATPng['elevation'], color='green', line_width=2, legend_label='ATPng Target Elevation')
# Velocities
p.extra_y_ranges = {'Velocity': Range1d(start=-0.1, end=0.1)}
p.add_layout(LinearAxis(y_range_name='Velocity', axis_label='Velocity'), 'right')
p.line(x=commanded_vel_el.index.values, y=commanded_vel_el['elevationVelocity'], color='black', alpha=0.7, y_range_name='Velocity', legend_label='ATMCS Commanded Elevation Velocity Trajectory')
p.line(x=measured_vel_el.index.values, y=measured_vel_el['elevationMotorVelocity'], color='red', alpha=0.5, y_range_name='Velocity', legend_label='ATMCS Measured Elevation Velocity Motor 1')
p.x(x=pd.to_datetime(commanded_vel_el_ATPng['private_sndStamp'], unit='s'), y=commanded_vel_el_ATPng['elevationVelocity'], y_range_name='Velocity', color='green', line_width=2, legend_label='ATPng Target Elevation Velocity')

p.legend.location = 'top_left'
p.legend.click_policy = 'hide'
show(p)

# Now look at Nasmyth 1

In [23]:
# Measured Position
nas1 = await get_data_frame('nasmyth1CalculatedAngle', 'lsst.sal.ATMCS.mount_Nasmyth_Encoders')

In [24]:
# Commanded Trajectory
commanded_nas1 = await get_data_frame('nasmyth1RotatorAngle', 'lsst.sal.ATMCS.trajectory')

In [25]:
# Pointing Kernel Command
commanded_nas1_ATPng = await client.query(f'SELECT "nasmyth1RotatorAngle", "private_sndStamp", "private_rcvStamp" FROM "efd"."autogen"."lsst.sal.ATMCS.command_trackTarget" WHERE {time_span}')

## Plot commanded position (by pointing component), target position by ATMCS, Calculated position by ATMCS

In [26]:
yr_cen=np.median(commanded_nas1['nasmyth1RotatorAngle'])
#yr_cen=0
dy=2.2
p = figure(x_axis_type='datetime', y_range=(yr_cen-dy, yr_cen+dy), plot_width=800, plot_height=400)
p.yaxis.axis_label = "Nasmyth Position (degrees)"
p.xaxis.axis_label = "Time"
p.line(x=commanded_nas1.index.values, y=commanded_nas1['nasmyth1RotatorAngle'], color='black', line_width=2, legend_label='ATMCS Commanded Nas1 Trajectory')
p.cross(x=nas1.index.values, y=nas1['nasmyth1CalculatedAngle'], color='red', line_width=2, line_dash='dashed', legend_label='ATMCS Nas1 Measured Nas1 Position')
p.cross(x=pd.to_datetime(commanded_nas1_ATPng['private_sndStamp'], unit='s'), y=commanded_nas1_ATPng['nasmyth1RotatorAngle'], color='green', line_width=5, legend_label='ATPng Target Nas1')

p.legend.location = 'bottom_left'
p.legend.click_policy = 'hide'
show(p)

## Look at Nasmyth motor velocities

In [27]:
#Measured
measured_vel_nas1 = await get_data_frame('nasmyth1MotorVelocity', 'lsst.sal.ATMCS.measuredMotorVelocity')
# Commanded
commanded_vel_nas1 = await get_data_frame('nasmyth1RotatorAngleVelocity', 'lsst.sal.ATMCS.trajectory')
# From Pointing
commanded_vel_nas1_ATPng = await client.query(f'SELECT "nasmyth1RotatorAngleVelocity", "private_sndStamp", "private_rcvStamp" FROM "efd"."autogen"."lsst.sal.ATMCS.command_trackTarget" WHERE {time_span}')

In [28]:
# Positions
p = figure(x_axis_type='datetime', y_range=(yr_cen-dy, yr_cen+dy), plot_width=800, plot_height=400)
p.line(x=commanded_nas1.index.values, y=commanded_nas1['nasmyth1RotatorAngle'], color='black', line_width=2, legend_label='ATMCS Commanded Nas1 Trajectory')
p.cross(x=nas1.index.values, y=nas1['nasmyth1CalculatedAngle'], color='red', line_width=2, line_dash='dashed', legend_label='ATMCS Nas1 Measured Nas1 Position')
p.cross(x=pd.to_datetime(commanded_nas1_ATPng['private_sndStamp'], unit='s'), y=commanded_nas1_ATPng['nasmyth1RotatorAngle'], color='green', line_width=2, legend_label='ATPng Target Nas1')
# Velocities
p.extra_y_ranges = {'Velocity': Range1d(start=-0.1, end=0.1)}
p.add_layout(LinearAxis(y_range_name='Velocity', axis_label='Velocity'), 'right')
p.line(x=commanded_vel_nas1.index.values, y=commanded_vel_nas1['nasmyth1RotatorAngleVelocity'], color='black', alpha=0.5, y_range_name='Velocity', legend_label='ATMCS Commanded Nas1 Velocity Trajectory')
p.line(x=measured_vel_nas1.index.values, y=measured_vel_nas1['nasmyth1MotorVelocity'], color='red', alpha=0.5, y_range_name='Velocity', legend_label='ATMCS Measured Nas1 Velocity Motor 1')
p.x(x=pd.to_datetime(commanded_vel_nas1_ATPng['private_sndStamp'], unit='s'), y=commanded_vel_nas1_ATPng['nasmyth1RotatorAngleVelocity'], y_range_name='Velocity', color='green', line_width=2, legend_label='ATPng Target Nas1 Velocity')

p.legend.location = 'top_left'
p.legend.click_policy = 'hide'
show(p)